<a href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/Precomputed_CHEMBERT_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Running the best GPS with KNN Generate**
<a href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/Precomputed_CHEMBERT_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Colab setup**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [2]:
!git clone https://github.com/raphaelrubrice/BornToOverfit.git
%cd BornToOverfit
!git checkout raph

Cloning into 'BornToOverfit'...
remote: Enumerating objects: 731, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 731 (delta 35), reused 35 (delta 24), pack-reused 674 (from 1)
Receiving objects: 100% (731/731), 1.06 MiB | 3.54 MiB/s, done.
Resolving deltas: 100% (500/500), done.
/content/BornToOverfit
Branch 'raph' set up to track remote branch 'raph' from 'origin'.
Switched to a new branch 'raph'


Installing requirements

In [3]:
!pip install -r data_baseline/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 55.3 MB/s eta 0:00:00


**Downloading the data**

In [4]:
!cp -r /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/data /content/BornToOverfit/data_baseline/.

## **KNN Generate**

Add Molecule cards

In [5]:
!python data_baseline/data_utils.py

BUILD + SAVE: mol_card for all graphs

[1/3] Loading: /content/BornToOverfit/data_baseline/data/train_graphs.pkl
Loaded graphs: 31008
Adding Molecule Cards..: 100% 31008/31008 [01:13<00:00, 423.09it/s]
Added/updated mol_card for: 31008
[2/3] Saving back to same location: /content/BornToOverfit/data_baseline/data/train_graphs.pkl
[3/3] Reloading to verify mol_card persisted: /content/BornToOverfit/data_baseline/data/train_graphs.pkl

--- Sample mol_card (first graph) ---
[MOL_FEATURES]
SMILES: NC1=C2N=CN([C@@H]3O[C@H](COP(=O)([O-])OP(=O)([O-])OC4O[C@H](CO)[C@@H](O)[C@H]4O)[C@@H](O)[C@H]3OP(=O)([O-])[O-])C2=NC=N1
atoms_total: 40
elements: C=15 N=5 O=17 P=3
molecular_weight: 635.265000
high_natural_abundance_atoms: 25
nuclear_spins: 1:5; 1/2:3
halogens_total: 0
metals_or_other_total: 0
formal_charge_net: -4
aromatic_atoms: 9
ring_atoms: 19
undirected_bonds: 43
bond_types: SINGLE=30 DOUBLE=3 AROMATIC=10
conjugated_bonds: 11
hybridization: SP2=13 SP3=27
chirality_centers: 7
radical_atoms: 0

Load embeddings and models

In [6]:
!cp -r /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/Cam_Raph /content/BornToOverfit/data_baseline/.

In [7]:
import os
path_to_folder = "/content/BornToOverfit/data_baseline/Cam_Raph/"
path_to_prototype = "/content/BornToOverfit/prototype/"

os.environ['SAVING_FOLDER'] = path_to_prototype + "FULL_model_v3_compat_from_dual_hd256/"
os.environ['SAVING_SUBFOLDER'] = "knn_llm"
os.environ['SAVING_SFT_SUBFOLDER'] = os.environ['SAVING_SUBFOLDER'] + "_sft"
os.environ['GNN_PATH'] = path_to_folder + "model_v3_compat_from_dual_hd256.pt"
os.environ['TRAIN_EMB_PATH'] = path_to_folder + "train_embeddings_RealChemBERT.pt"
os.environ['VAL_EMB_PATH'] = path_to_folder + "validation_embeddings_RealChemBERT.pt"

Create merged TRAIN/VAL data

In [8]:
os.environ['DATA_FOLDER'] = "/content/BornToOverfit/data_baseline/data"
os.environ['MERGED_GRAPHS'] = os.environ['DATA_FOLDER'] + "/train_val_graphs.pkl"
!python data_baseline/merge_embeddings.py \
  --train_path $TRAIN_EMB_PATH \
  --val_path $VAL_EMB_PATH \
  --output_dir $DATA_FOLDER

Loading embeddings from /content/BornToOverfit/data_baseline/Cam_Raph/train_embeddings_RealChemBERT.pt...
 -> Loaded 31008 embeddings.
Loading embeddings from /content/BornToOverfit/data_baseline/Cam_Raph/validation_embeddings_RealChemBERT.pt...
 -> Loaded 1000 embeddings.
Max Train ID found: 31007. Offset calculated: 31008
Re-indexing 1000 validation embeddings with offset +31008...
Merged 31008 + 1000 = 32008 total embeddings.
Saving 32008 embeddings to /content/BornToOverfit/data_baseline/data/train_val_embeddings.pt...
Done.


In [9]:
os.environ['MERGED_EMB'] = os.environ['DATA_FOLDER'] + "/train_val_embeddings.pt"
!python data_baseline/merge_embeddings.py \
  --train_path $TRAIN_EMB_PATH \
  --val_path $VAL_EMB_PATH \
  --output_dir $DATA_FOLDER

Loading embeddings from /content/BornToOverfit/data_baseline/Cam_Raph/train_embeddings_RealChemBERT.pt...
 -> Loaded 31008 embeddings.
Loading embeddings from /content/BornToOverfit/data_baseline/Cam_Raph/validation_embeddings_RealChemBERT.pt...
 -> Loaded 1000 embeddings.
Max Train ID found: 31007. Offset calculated: 31008
Re-indexing 1000 validation embeddings with offset +31008...
Merged 31008 + 1000 = 32008 total embeddings.
Saving 32008 embeddings to /content/BornToOverfit/data_baseline/data/train_val_embeddings.pt...
Done.


Use all data ?

In [10]:
USE_ALL_DATA = False
if USE_ALL_DATA:
    os.environ['TRAINING_DATA'] = os.environ['MERGED_EMB']
else:
    os.environ['TRAINING_DATA'] = os.environ['TRAIN_EMB_PATH']

Fine-tune the LLM with SFT

In [11]:
os.environ['STEPS'] = '500'
os.environ['MAX_LENGTH'] = '2048'

In [ ]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_finetune \
  --do_sft \
  --k 5 \
  --use_lora \
  --sft_steps $STEPS \
  --sft_lora 128 \
  --sft_neftune \
  --per_device_bs 1 \
  --grad_accum 8 \
  --base_llm deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
  --out_llm_dir $SAVING_SUBFOLDER \
  --max_prompt_length $MAX_LENGTH

2026-01-11 22:22:36.200937: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-11 22:22:36.216846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768170156.238198    5289 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768170156.244637    5289 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768170156.260951    5289 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

GRPO


In [14]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 89.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
os.environ['SFT_SAVING_FOLDER'] = "/content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/FULL_model_v3_compat_from_dual_hd256/knn_llm_sft"
os.environ['RL_SAVING_FOLDER'] = os.environ['SAVING_SUBFOLDER'] + "_rl"

!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_finetune \
  --rl_algo grpo \
  --k 5 \
  --use_lora \
  --rl_steps $STEPS \
  --bf16 \
  --per_device_bs 1 \
  --num_generations 4 \
  --grad_accum 8 \
  --base_llm $SFT_SAVING_FOLDER \
  --out_llm_dir $RL_SAVING_FOLDER \
  --max_prompt_length $MAX_LENGTH

2026-01-12 15:30:30.630984: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-12 15:30:30.651244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768231830.673726   24887 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768231830.680364   24887 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768231830.698226   24887 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Generate on validation with the fine-tuned model + evaluation metrics

In [19]:
os.environ['SFT_SAVING_FOLDER'] = "/content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/FULL_model_v3_compat_from_dual_hd256/knn_llm_sft"
os.environ['NEW_SAVING_FOLDER'] = "/content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/FULL_model_v3_compat_from_dual_hd256/knn_llm_sft_penalized_rep"
os.environ['SAVING_FOLDER'] = "/content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/FULL_model_v3_compat_from_dual_hd256/"

In [ ]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_generate --split validation --k 5 \
  --gen_batch_size 8 \
  --base_llm $SFT_SAVING_FOLDER \
  --out_llm_dir $SFT_SAVING_FOLDER \
  --max_prompt_length $MAX_LENGTH

2026-01-12 21:24:37.126780: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-12 21:24:37.145093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768253077.167421    6694 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768253077.174222    6694 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768253077.191376    6694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Generate on test (no references; no evaluation)

In [ ]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_emb_path $TRAINING_DATA \
  --do_generate --split test --k 5 \
  --gen_batch_size 8 \
  --out_llm_dir $SAVING_SFT_SUBFOLDER \
  --max_prompt_length $MAX_LENGTH

2026-01-12 07:53:47.585782: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-12 07:53:47.603212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768204427.624439  152392 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768204427.630954  152392 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768204427.647200  152392 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

See generated data on the test set

In [ ]:
import pandas as pd
os.environ['FINAL_SAVING_FOLDER'] = os.environ['SAVING_FOLDER'][:-1]
test = pd.read_csv(os.environ['FINAL_SAVING_FOLDER'] + "/test_knn_gen_k5.csv")
test

,ID,Prompt,generated_description,NN1,NN2,NN3,NN4,NN5
0,130,You are an expert chemist. Generate a factual ...,The molecule is the stable isotope of zinc wit...,14677,2289,NaN,NaN,NaN
1,615,You are an expert chemist. Generate a factual ...,The molecule is an organic heteroatom that con...,14484,17988,NaN,NaN,NaN
2,103,You are an expert chemist. Generate a factual ...,The molecule is the stable isotope of yttrium ...,27655,2289,NaN,NaN,NaN
3,119,You are an expert chemist. Generate a factual ...,The molecule is an alkane that is octane subst...,12729,22545,NaN,NaN,NaN
4,404,You are an expert chemist. Generate a factual ...,The molecule is an organosulfur heterocycle th...,5290,27029,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
995,14,You are an expert chemist. Generate a factual ...,The molecule is a pseudoguaianolide with anti-...,2601,22872,1.0,20660.0,7049.0
996,751,You are an expert chemist. Generate a factual ...,"The molecule is a diterpenoid that is 6a,7,8,1...",11015,19168,10235.0,22987.0,13686.0
997,998,You are an expert chemist. Generate a factual ...,The molecule is an L-alpha-amino acid ester re...,11088,23286,20439.0,20537.0,3658.0
998,995,You are an expert chemist. Generate a factual ...,The molecule is an alkyl cinnamate obtained by...,28494,6409,3907.0,685.0,2544.0


In [ ]:
print(test.iloc[18]["Prompt"])
print(test.iloc[18]["generated_description"])

You are an expert chemist. Generate a factual description of the [QUERY MOLECULE] strictly based on its feature card.

**Context Guidelines:**
* **[STRUCTURAL TEMPLATES]:** Use these molecules to determine sentence structure, IUPAC nomenclature style, and general scaffold description. **CAUTION:** Even if high similarity, specific atoms or groups may differ. Do NOT infer chemical properties (e.g., bond counts, specific substituents) from templates unless they exist in the Query Card.
* **[FUNCTIONAL REFERENCES]:** Use these to identify potential abstract biomedical roles (e.g., 'anti-bacterial', 'metabolite'). **CAUTION:** These may be chemically distant. **NEVER** copy physical numbers (Mass, Spin) or specific bio-activity data unless supported by the [QUERY MOLECULE] structure.

**Strict Constraints:**
1. The [QUERY MOLECULE] Card is the ABSOLUTE GROUND TRUTH.
2. If a neighbor contradicts the Query, IGNORE the neighbor.
3. Never mention 'Neighbor' or 'Template' in the final output.



Saving to Drive

In [ ]:
!cp -r $FINAL_SAVING_FOLDER /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/.